In [ ]:
# # Importa biblioteca para utilização de ferramentas de sistema operacional
# import os

# # Clona o repositório, com os datasets, se não existir uma versão dele
# if not os.path.exists('/content/iia-trabalho-1'):
#   !git clone https://github.com/smmstakes/iia-trabalho-1.git

# # Instalando biblioteca para geração de dados falsos
# !pip install Faker

In [19]:
# Importa todas as bibliotecas a serem utilizadas no projeto
import time
import json
import pickle
import unicodedata

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

from faker import Faker
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

In [ ]:
# # Lista com as cidades dos produtores
with open("./data/json/cities_list.json", "r") as file:
    cities_list = json.load(file)

# Dicionário com uma tupla com a localização de cada cidade
# locations = get_locations(cities_list)
with open("./data/json/locations.json", "r") as file:
  locations = json.load(file)

with open("./data/json/producers_locations.json", "r") as file:
  producers_location = json.load(file)

with open("./data/json/producers_ra.json", "r") as file:
    producers_ra = json.load(file)

# Lista de cada produto ofertado pelos produtores
with open("./data/json/products_list.json", "r") as file:
    products_list = json.load(file)

In [ ]:
# Definindo funções auxiliares

def get_locations(cities_list: list) -> dict:
  # Definindo um agente para pegar as coordenadas
  geolocator = Nominatim(user_agent="loc_producers", timeout=10)
  locations = {}

  for city in cities_list:
    try:
      time.sleep(1)

      # Requisição para uma cidade na lista de cidade
      loc = geolocator.geocode(f"{city}, DF, Brasil")
      locations[city] = (loc.latitude, loc.longitude) if loc else (None, None)

    except Exception as e:
      print(f"Erro ao tentar achar: {city}: {e}")
      locations[city] = (None, None)

  return locations

def name_formatter(name: str) -> str:
    # Remove acentos e normaliza caracteres
    new_name = unicodedata.normalize('NFKD', name)

    # Substitui espaços por underscores e remove caracteres especiais
    return new_name.encode('ascii', 'ignore').decode('ascii').replace(' ', '_')

def producer_infos(producer: str) -> tuple:
  try:
    prod, formatted_local = producer.split("_", 1)

    if prod.startswith("Rede"):
      return "Rede Terra" ,"Santa Maria", locations["Santa Maria"][0], locations["Santa Maria"][1]

    for city in cities_list:
      if name_formatter(city) == formatted_local:
        correct_location = city
        lat, lon = locations[correct_location]
        return prod, correct_location, lat, lon

  except ValueError as e:
      return "Desconhecido", "Desconhecido", 0.0, 0.0
  
def producer_name_formatter(producers: list) -> list:
    prods_formatted = []
    for producer, locals in producers.items():
        for ra in locals:
            produtor_formatado = producer.replace(' ', '_')
            ra = name_formatter(ra)
            prods_formatted.append(f"{produtor_formatado}_{ra}")
    return prods_formatted

def get_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

In [ ]:
# Concatenando nome do produtor com suas RAs

# producers_formatted = producer_name_formatter(producers_ra)
with open("./data/json/producers_formatted.json", "r") as file:
    producers_formatted =  json.load(file)

In [ ]:
# Definindo número inicial para geração pseudo-aleatória
np.random.seed(53)

matrix_size = 2000

# Base do DataFrame de produtos
products = {
    "produto": [],
    "organico": [],
    "nome_produtor": [],
    "local": [],
    "latitude": [],
    "longitude": []
}

# Adicionando informações sobre produtos vendidos por cada produtor
for _ in range(matrix_size):
    products["produto"].append(np.random.choice(products_list))
    products["organico"].append(np.random.choice([0, 1]))

    prod = np.random.choice(producers_formatted)
    producer, local, lat, lon = producer_infos(prod)

    products["nome_produtor"].append(producer)
    products["local"].append(local)
    products["latitude"].append(lat)
    products["longitude"].append(lon)

# Gerando DataFrame e exportando para csv
df_products = pd.DataFrame(products).drop_duplicates()
df_products.to_csv('./data/datasets/producers.csv', index=False)
df_products.head(20)

In [ ]:
# Gerador de dados falsos
fake = Faker()

n_reviews = 7500
n_users = 2500

# Criando usuários ficticios
users = [fake.uuid4() for _ in range(n_users)]

# Basa para o DataFrame de avaliação de produtos
reviews = {
          "id_usuario": [],
          "produto": [],
          "organico": [],
          "nome_produtor": [],
          "local": [],
          "avaliacao": []
        }

for _ in range(n_reviews):
  reviews["id_usuario"].append(np.random.choice(users))

  id_produto = np.random.choice(len(df_products))

  reviews["produto"].append(df_products.iloc[id_produto]["produto"])
  reviews["organico"].append(df_products.iloc[id_produto]["organico"])
  reviews["nome_produtor"].append(df_products.iloc[id_produto]["nome_produtor"])
  reviews["local"].append(df_products.iloc[id_produto]["local"])
  reviews["avaliacao"].append(np.random.randint(1, 6))

df_reviews = pd.DataFrame(reviews).drop_duplicates()
df_reviews.to_csv('./data/datasets/reviews.csv', index=False)
df_reviews.head(20)

In [ ]:
utility_matrix = pd.pivot_table(
  df_reviews,
  values='avaliacao',
  index='nome_produtor',
  columns='produto',
  fill_value=0
  ).round(2)

utility_matrix.to_csv('./data/datasets/matrix_reviews.csv')
utility_matrix.head(20)

In [ ]:
# Juntando os dados para formar uma base de recomendação

# 1. Juntar as avaliações com os dados dos produtores
df_full_reviews = df_reviews.merge(df_products, on=["nome_produtor", "produto", "organico", "local"], how="inner")

df_full_reviews.to_csv("./data/datasets/df_full_reviews.csv", index=False)

# Inicializar encoders
le_usuario = LabelEncoder()
le_produto = LabelEncoder()
le_produtor = LabelEncoder()
le_local = LabelEncoder()

# Codificar colunas categóricas
df_full_reviews["usuario_id"] = le_usuario.fit_transform(df_full_reviews["id_usuario"])
df_full_reviews["produto_id"] = le_produto.fit_transform(df_full_reviews["produto"])
df_full_reviews["produtor_id"] = le_produtor.fit_transform(df_full_reviews["nome_produtor"])
df_full_reviews["local_id"] = le_local.fit_transform(df_full_reviews["local"])

df_full_reviews[["usuario_id", "produto_id", "organico", "avaliacao", "produtor_id", "local_id", "latitude", "longitude"]].head(20)

# Features usadas para similaridade (excluindo avaliação que será alvo)
feature_cols = ["produto_id", "organico", "produtor_id", "local_id", "latitude", "longitude"]

# Matriz de características e vetor de alvo
X = df_full_reviews[feature_cols].values
y = df_full_reviews['avaliacao'].values

# Modelo de vizinhos mais próximos
knn_model = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn_model.fit(X)

In [ ]:
def get_recommendation_candidates(desired_products, producer, location):
    # global df_full_reviews

    if isinstance(desired_products, str):
        desired_products = [desired_products]

    candidates = df_full_reviews[
        (df_full_reviews['produto'].isin(desired_products)) |
        (df_full_reviews['nome_produtor'] == producer) |
        (df_full_reviews['local'] == location)
    ].copy()

    # Remove combinações exatas
    candidates = candidates[~(
        (candidates['produto'].isin(desired_products)) &
        (candidates['nome_produtor'] == producer) |
        (candidates['produto'].isin(desired_products))
    )]

    return candidates


def normalize_distance(candidates, latitude, longitude):
    candidates['distancia_km'] = candidates.apply(
        lambda row: get_distance((latitude, longitude), (row['latitude'], row['longitude'])),
        axis=1
    )
    max_dist = candidates['distancia_km'].max()
    candidates['proximidade'] = 1 - (candidates['distancia_km'] / max_dist)
    return candidates


def calculate_average_rating(candidates):
    candidates['media_produtor_produto'] = (
        candidates.groupby(['produto', 'nome_produtor'])['avaliacao']
        .transform('mean')
    )
    candidates['avaliacao_norm'] = candidates['media_produtor_produto'] / 5.0
    return candidates


def calculate_score(recommendation_type: int, is_organic: int, feature_values: list) -> float:
    # Inicialização dos pesos padrão
    weights = {
        "rating": 0.5,
        "proximity": 0.5,
        "organic": 0.0
    }

    if recommendation_type in (0, 2):
        if is_organic == 1:
            weights.update({
                "rating": 0.3,
                "proximity": 0.5,
                "organic": 0.2
            })
        else:
            weights["organic"] = -1.0

    elif recommendation_type == 1:
        weights.update({
            "rating": 0.7,
            "proximity": 0.3,
            "organic": 0.0
        })
        # Para tipo 1, só rating e proximidade são usados
        return (
            weights["rating"] * feature_values[0]
            + weights["proximity"] * feature_values[1]
        )

    # Cálculo do score final para tipos 0 e 2
    return (
        weights["rating"] * feature_values[0]
        + weights["proximity"] * feature_values[1]
        + weights["organic"] * feature_values[2]
    )

In [ ]:
def recommend_best_products(desired_products, producer, location, organic, latitude, longitude):
    candidates = get_recommendation_candidates(desired_products, producer, location)

    if candidates.empty:
        return pd.DataFrame({'mensagem': ['Nenhuma recomendação alternativa encontrada.']})

    candidates = normalize_distance(candidates, latitude, longitude)
    candidates = calculate_average_rating(candidates)

    features_values = [candidates['avaliacao_norm'], candidates['proximidade'], candidates['organico']]
    candidates["score"] = calculate_score(0, organic, features_values)

    top_recommendations = (
        candidates.sort_values(by='score', ascending=False)
        .drop_duplicates(subset=['produto', 'nome_produtor'])
        .head(5)
    )

    return top_recommendations[[
        'produto', 'nome_produtor', 'local', 'organico',
        'media_produtor_produto', 'distancia_km', 'score'
    ]].round(2)

In [ ]:
# Recomenda os melhores produtos de determinado produtor com base no produto fornecido e se é organico
recommend_best_products(
    desired_products=['Uva', "Limão"],
    producer='Asphor',
    location='Gama',
    organic=0,
    latitude=-16.0170857,
    longitude=-48.0653054
)

In [ ]:
def get_producer_recomendation(df_reviews, product):
    candidates = df_reviews[df_reviews['produto'] == product].copy()

    if product.strip() == "":
        candidates = df_reviews.copy()

    if candidates.empty:
        return pd.DataFrame({'mensagem': ['Nenhum produtor encontrado para este produto.']})
    
    # candidates = candidates.sort_values('media_avaliacao', ascending=False)
    return candidates.drop_duplicates(subset=['nome_produtor'], keep='first')

def calculate_average_producer_rating(candidates):
    candidates = (
        candidates.groupby(['nome_produtor', 'local', 'latitude', 'longitude', 'organico'])
        .agg(media_avaliacao=('avaliacao', 'mean'))
        .reset_index()
    )
    candidates['avaliacao_norm'] = candidates['media_avaliacao'] / 5.0
    return candidates

In [ ]:
def recommend_best_productors(product, latitude, longitude, top_n=5):
    candidates = get_producer_recomendation(df_full_reviews, product)
    candidates = calculate_average_producer_rating(candidates)
    candidates = normalize_distance(candidates, latitude, longitude)

    features_values = [candidates['avaliacao_norm'], candidates['proximidade'], candidates["organico"]]
    candidates["score"] = calculate_score(1, 0, features_values)

    top_result = candidates.sort_values(by='score', ascending=False).head(top_n)

    return top_result[[
        'nome_produtor', 'local', 'organico',
        'media_avaliacao', 'distancia_km', 'score'
    ]].round(2)

In [ ]:
# Recomenda os melhores produtores de determinado produto considerando a localização como foco
recommend_best_productors(
    product='Maracujá',
    latitude=-15.7183687,
    longitude=-47.9950273
)

In [ ]:
def get_products_recomendation(df_full_reviews, producer, unwanted_products):
    if unwanted_products is None:
        unwanted_products = []

    candidates = df_full_reviews[
        (df_full_reviews['nome_produtor'] == producer) &
        (~df_full_reviews['produto'].isin(unwanted_products))
    ].copy()

    if candidates.empty:
        return pd.DataFrame({'mensagem': ['Nenhum produtor encontrado para este produto.']})
    
    return candidates


In [ ]:
def recommend_best_product_productors(producer, local, organic, latitude, longitude, unwanted_products=None):

    candidates = get_products_recomendation(df_full_reviews, producer, unwanted_products)
    candidates = normalize_distance(candidates, latitude, longitude)
    candidates = candidates.drop_duplicates(subset=['produto', 'nome_produtor'])
    candidates = calculate_average_rating(candidates)

    features_values = [candidates['avaliacao_norm'], candidates['proximidade'], candidates['organico']]
    candidates["score"] = calculate_score(2, organic, features_values)
    
    resultado = candidates.sort_values(by='score', ascending=False).head(5)

    return resultado[['produto', 'nome_produtor', 'local', 'organico',
                      'media_produtor_produto', 'distancia_km', 'score']]


In [ ]:
# Recomenda os melhores produtos de um determinado produtor

recommend_best_product_productors(
    producer='Cooperbrasília',
    local='Gama',
    organic=0,
    latitude=-15.650053,
    longitude=-47.784845,
    unwanted_products=["Mandioca", "Morango"]
)

In [20]:
full_data = {
    "model": knn_model
}

with open("./data/model/model.pkl", "wb") as file:
    pickle.dump(full_data, file)